# Peer Review: Creating a Web-Service

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')

from fastapi import FastAPI
from fastapi.responses import HTMLResponse
from fastapi.responses import JSONResponse
from starlette.staticfiles import StaticFiles

In [2]:
app = FastAPI()

def solve(a, b, c):
    D = b**2 - 4*a*c
    if D > 0:
        return [(-b + np.sqrt(D))/(2*a), (-b - np.sqrt(D))/(2*a)]
    elif D == 0:
        return [-b/(2*a)]
    else:
        return []

@app.get("/solve", response_class=JSONResponse)
async def read_item(a: int, b: int, c: int):
    return {"roots": solve(a, b, c)}

@app.get("/main", response_class=HTMLResponse)
async def read_item():
    return """
    <html>
        <head>
            <title>Quadratic equation solver</title>
        </head>
        <body>
            <h1>Quadratic equation solver</h1>
            <form action="/solve" method="get">
                <label for="a">a:</label>
                <input type="text" id="a" name="a"><br><br>
                <label for="b">b:</label>
                <input type="text" id="b" name="b"><br><br>
                <label for="c">c:</label>
                <input type="text" id="c" name="c"><br><br>
                <input type="submit" value="Submit">
            </form>
            <form action="/plot" method="get">
                <label for="a">a:</label>
                <input type="text" id="a" name="a"><br><br>
                <label for="b">b:</label>
                <input type="text" id="b" name="b"><br><br>
                <label for="c">c:</label>
                <input type="text" id="c" name="c"><br><br>
                <input type="submit" value="Plot">
            </form>
        </body>
    </html>
    """

@app.get("/plot", response_class=HTMLResponse)
async def read_item(a: int, b: int, c: int):
    roots = solve(a, b, c)
    if len(roots) == 0:
        return "No roots"
    elif len(roots) == 1:
        x = np.linspace(roots[0]-1, roots[0]+1, 100)
        y = a*x**2 + b*x + c
        plt.plot(x, y)
    else:
        x1 = np.linspace(roots[0]-1, roots[0]+1, 100)
        y1 = a*x1**2 + b*x1 + c
        x2 = np.linspace(roots[1]-1, roots[1]+1, 100)
        y2 = a*x2**2 + b*x2 + c
        plt.plot(x1, y1)
        plt.plot(x2, y2)
    plt.savefig("plot.png")
    return """
    <html>
        <head>
            <title>Quadratic equation plot</title>
        </head>
        <body>
            <h1>Quadratic equation plot</h1>
            <img src="plot.png" alt="rtt">
        </body>
    </html>
    """

### Trying the app

In [3]:
import nest_asyncio
nest_asyncio.apply()
from fastapi.testclient import TestClient

client = TestClient(app)

response = client.get("/solve?a=1&b=-4&c=3")
print(response.json())

{'roots': [3.0, 1.0]}
